# Práctico N° 2 Análisis de Grafos

Integrantes: Nicolás Benjamín Ocampo, Antonela Sambuceti

En este práctico, trabajaremos con un dataset extraído de Twitter. La idea es emplear los conceptos de grafos vistos en clase sobre un caso real de actualidad.

## Dataset

El dataset consiste en un conjunto de hilos de tweets, con un total de ~150000 tweets, extraídos entre Enero y Marzo de 2021. La temática de los mismos está referida a la vacunación contra el covid-19 en Argentina.

Pueden descargar el dataset del siguiente [link](https://drive.google.com/file/d/1X_qKsE8muAnom2tDX4sLlmBAO0Ikfe_G/view?usp=sharing).

### Campos

- **created_at:** Fecha del tweet
- **id_str:** ID del tweet
- **full_text:** Contenido del tweet
- **in_reply_to_status_id:** ID del tweet inmediatamente anterior en el hilo
- **in_reply_to_user_id:** Autor del tweet inmediatamente anterior en el hilo
- **user.id:** Autor del tweet
- **user_retweeters:** Lista de ID de usuarios que retweetearon el tweet
- **sentiment:** Etiquetado manual que indica el sentimiento o intención del tweet con respecto al tweet anterior en el hilo

## Configuración inicial

In [1]:
import pandas as pd
import numpy as np
import seaborn as sn
import matplotlib.pyplot as plt
import math
from cdlib import NodeClustering, evaluation, algorithms
import igraph as ig
import networkx as nx

from nltk import (corpus, tokenize, download)
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
download("stopwords")
download('punkt')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\anto_\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\anto_\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

### Cargamos el dataset

In [2]:
DATASET_FILE = "dataset/vacunas.csv"

dtypes = {
    'id_str': str,
    'full_text': str,
    'in_reply_to_status_id': str,
    'in_reply_to_user_id': str,
    'user.id': str
}
df = pd.read_csv(DATASET_FILE, dtype=dtypes).dropna(subset=['user_retweeters'])
df['user_retweeters'] = df['user_retweeters'].apply(lambda x: [str(elem) for elem in eval(x)])
print(df.shape)

(155123, 8)


### Construcción del grafo

In [3]:
users = df["user.id"].unique().tolist()
edges = []
for _, (user_id, retweet_ids) in df[["user.id", "user_retweeters"]].iterrows():
    retweeters = [(user_id, rt) for rt in retweet_ids]
    edges.extend(retweeters)

In [4]:
#calculamos grafo no dirigido
G = nx.Graph()
G.add_nodes_from(users)
G.add_edges_from(edges)

print(nx.info(G))

Graph with 65934 nodes and 93404 edges


Calculamos la componente conexa más grande

In [5]:
largest_cc = max(nx.connected_components(G), key=len)

In [6]:
larg_G = nx.Graph()
larg_G.add_nodes_from(largest_cc)
larg_G.add_edges_from(edges)

print(nx.info(larg_G))

Graph with 39800 nodes and 93404 edges


In [7]:
df_user = pd.DataFrame({'user':list(larg_G.nodes())})

In [8]:
deg_seq = np.array([k for v, k in larg_G.degree()])
df_user['degree'] = deg_seq

### Cálculo de Medidas de Centralidad

In [9]:
#generados grafo para libreria igraph
g_ig = ig.Graph.TupleList(larg_G.edges())
print(g_ig.summary())

IGRAPH UN-- 39800 93404 -- 
+ attr: name (v)


In [10]:
betweenness = g_ig.betweenness()
eigenvector = nx.eigenvector_centrality(larg_G,max_iter=1000)
pagerank = nx.pagerank(larg_G)
clustering_coefficient =  nx.clustering(larg_G)

In [11]:
df_user['betweenness'] = betweenness
df_user['eigenvector'] = list(eigenvector.values())
df_user['pagerank'] = list(pagerank.values())
df_user['Cws'] = list(clustering_coefficient.values())

df_user.describe().round(2)

,degree,betweenness,eigenvector,pagerank,Cws
count,39800.00,3.980000e+04,39800.00,39800.00,39800.00
mean,4.69,5.310058e+04,0.00,0.00,0.10
std,69.30,2.036286e+06,0.00,0.00,0.24
min,1.00,0.000000e+00,0.00,0.00,0.00
25%,1.00,0.000000e+00,0.00,0.00,0.00
50%,1.00,0.000000e+00,0.00,0.00,0.00
75%,3.00,2.133930e+03,0.00,0.00,0.00
max,8207.00,2.703364e+08,0.36,0.04,1.00


In [12]:
columns=['degree','eigenvector','pagerank','Cws', 'betweenness']
data = {}
for col in columns:
    top_users = df_user.nlargest(10, columns=[col])['user'].tolist()
    data[col] = top_users
pd.DataFrame(data)

,degree,eigenvector,pagerank,Cws,betweenness
0,252168075,130979339,252168075,297767050,141298600
1,130979339,73102744,130979339,449875232,876225889025085441
2,73102744,252168075,73102744,1364419256658591750,310377136
3,367933714,367933714,593189095,1169756487612358656,1277348497721876481
4,593189095,2687724840,367933714,982633981719072769,1324866284195024896
5,2687724840,593189095,2687724840,1026326278910234625,336706010
6,931564592328781824,312708081,144929758,1200475594410151936,4627808362
7,144929758,144929758,931564592328781824,67229075,1171822801113759750
8,312708081,931564592328781824,312708081,196442815,920082176
9,1077176953,959033548379508736,1077176953,3383952485,31058772


### Cálculo de Comunidades

#### Algoritmo de Louvain

Recalculamos las comunidades con el algoritmo de Louvain con resolution 2, para quedarnos con 2 grandes comunidades para analizar.

In [13]:
comms = algorithms.louvain(larg_G, resolution=2, randomize=False)

In [14]:
comm_df = pd.DataFrame({
    "communities": comms.communities,
    "comm_id": np.arange(0, len(comms.communities))
})

In [15]:
comm_df = comm_df.assign(comm_size=comm_df["communities"].apply(lambda c: len(c)))

## Actividades

### 4. Extracción de etiquetas
En el archivo etiquetas.csv están las etiquetas para un pequeño subconjunto de nodos. Podemos interpretar el valor de la etiqueta como la pertenencia a una determinada clase, donde los usuarios de una misma clase en general tienden a expresar apoyo entre sí.

In [18]:
DATASET_FILE = "dataset/etiquetas.csv"

df_etiq = pd.read_csv(DATASET_FILE, dtype=dtypes)
df_etiq= df_etiq.rename(columns={'user.id':'user'})

In [19]:
df_etiq.Clase.value_counts()

0    107
1     89
Name: Clase, dtype: int64

In [20]:
df_etiq = pd.merge(df_etiq, df_user, on='user',how='left')
df_etiq= df_etiq.dropna()

,user,Clase,degree,betweenness,eigenvector,pagerank,Cws
2,1329815581302853632,0,1.0,0.000000,0.002809,0.000008,0.000000
4,811607948,0,5.0,2132.671224,0.003172,0.000019,0.400000
5,405924380,0,72.0,2684.213654,0.007031,0.000243,0.046557
6,811960507829800960,0,7.0,16751.992698,0.000260,0.000031,0.000000
7,3108851163,0,41.0,0.000000,0.005029,0.000135,0.056098
...,...,...,...,...,...,...,...
190,1183974626,1,8.0,0.000000,0.012996,0.000039,0.500000
191,15750760,1,1.0,0.000000,0.000112,0.000008,0.000000
192,882731195834003457,1,1.0,0.000000,0.000179,0.000008,0.000000
193,164350783,1,8.0,0.000000,0.004351,0.000037,0.178571


In [44]:
df_etiq.Clase.value_counts()

0    84
1    68
Name: Clase, dtype: int64

#### 4.1 Usuarios referentes de cada clase 
Utilizar alguna medida de centralidad calculada sobre el grafo de retweets.

In [45]:
#Top para clase 0
columns=['degree','eigenvector','pagerank','Cws', 'betweenness']
data = {}
for col in columns:
    top_users = df_etiq[df_etiq['Clase'] == 0].nlargest(5, columns=[col])['user'].tolist()
    data[col] = top_users
pd.DataFrame(data)

,degree,eigenvector,pagerank,Cws,betweenness
0,252168075,252168075,252168075,2947569093,1180936849
1,510740590,510740590,510740590,140374618,510740590
2,145493922,145493922,43814969,172417422,202984292
3,484349869,484349869,145493922,973491850530164736,290671142
4,43814969,1272713534078619648,484349869,811607948,1290051460810629122


Obsevamos que el usuario 252168075, correspondiente a Norabar, que había sido seleccionado en el top10 de influencers en el enunciado número 2, también se encuentra en como principal influencer de la clase 0.

In [46]:
#Top para clase 1
columns=['degree','eigenvector','pagerank','Cws', 'betweenness']
data = {}
for col in columns:
    top_users = df_etiq[df_etiq['Clase'] == 1].nlargest(5, columns=[col])['user'].tolist()
    data[col] = top_users
pd.DataFrame(data)

,degree,eigenvector,pagerank,Cws,betweenness
0,73102744,73102744,73102744,910956876686381058,202824848
1,367933714,367933714,367933714,146631761,60939248
2,2687724840,2687724840,2687724840,1212718019849248768,1163194886352461824
3,3421061763,3421061763,3421061763,1183974626,1954572703
4,4160782817,4160782817,4160782817,2693592596,837186873143488514


Obsevamos que el usuario 73102744, correspondiente a Ernestorr, que había sido seleccionado en el top10 de influencers en el enunciado número 2, también se encuentra en como principal influencer de la clase 1.

#### 4.2 Utilizando los resultados del práctico anterior, determinar si los usuarios de cada clase forman parte de distintas comunidades

In [57]:
comm_df.columns

Index(['communities', 'comm_id', 'comm_size'], dtype='object')

In [68]:
com_1 = list(comm_df["communities"][0])
com_2 = list(comm_df["communities"][1])

In [71]:
clase_0 = list(df_etiq[df_etiq['Clase'] == 0]['user'])
clase_1 = list(df_etiq[df_etiq['Clase'] == 1]['user'])

In [77]:
#Cantidad de usuarios de la clase 1 que se encuentran en la comunidad 2
len([x for x in clase_0 if x in com_2])

84

In [78]:
#Cantidad de usuarios de la clase 0 que se encuentran en la comunidad 1
len([x for x in clase_1 if x in com_1])

65

Todos los usuarios de la clase 0 están presentes en la comunidad 2 y casi todos los usuarios de la clase 1 están presentes en la comunidad 1.

### Opcional: 
Reconstruir el archivo "etiquetas.csv". Para eso, hacer lo siguiente 
- Construir un grafo en donde los nodos sean usuarios, y donde los enlaces unan dos nodos si entre ellos hubo más respuestas de apoyo que de oposición.
- Extraer las dos componentes más grandes del grafo. Esos serán nuestros nodos etiquetados.

## 5. Embedding de nodos
Generar un embedding del grafo de retweets utilizando el algoritmo word2vec.
Reducir a 2 la dimensionalidad del embedding utilizando PCA y t-SNE.
Graficar los embeddings correspondientes a los datos etiquetados. ¿Es posible diferenciar unos de otros?

## Opcional: 
Graficar además los embeddings de los nodos que forman parte de las comunidades asociadas a cada clase. Determinar si el embedding permite distinguir cada comunidad.

## Opcional: 6. Redes neuronales de grafos
El archivo word_vectors.csv contiene un embedding de 300 dimensiones para cada tweet, otenido utilizando un modelo preentrenado de FastText. Construir una matriz de features para los nodos tomando, para cada usuario, el promedio de los vectores correspondientes a los tweets que escribió. Utilizando estos features, y tomando como ejemplos etiquetados los usuarios de "etiquetas.csv" entrenar una red neuronal de grafos para realizar una clasificación binaria sobre el resto de los nodos. Pueden utilizar como base el siguiente modelo:

In [ ]:
class GCN(torch.nn.Module):
    def __init__(self):
        super(GCN, self).__init__()
        torch.manual_seed(1234)
        self.conv1 = GCNConv(dataset.num_features, 4)
        self.conv2 = GCNConv(4, 4)
        self.conv3 = GCNConv(4, 2)
        self.classifier = Linear(2, dataset.num_classes)

    def forward(self, x, edge_index):
        h = self.conv1(x, edge_index)
        h = h.tanh()
        h = self.conv2(h, edge_index)
        h = h.tanh()
        h = self.conv3(h, edge_index)
        h = h.tanh()  # Embedding final
        
        # Aplicamos un clasificador lineal sobre el embedding
        out = self.classifier(h)

        return out, h

Observación: para alimentar la red neuronal, es necesario construir un objeto de la clase Dataset de PyTorch-Geometric. Una forma de hacer eso es la siguiente

In [ ]:
from torch_geometric.data import InMemoryDataset, Data

## Reemplazar por el grafo correspondiente
g = nx.Graph()

## Etiquetas. Reemplazar por las clases del archivo 'etiquetas.csv'.
## Asignar la clase '2' a los ejemplos no etiquetados
labels = [1, 0, 2, ..., 1]

## True si el ejemplo está etiquetado (clases 0 y 1)
train_idx = [True, True, False, ..., True]

## Matriz de features (word vectors)
features = ...

adj = nx.to_scipy_sparse_matrix(g).tocoo()
row = torch.from_numpy(adj.row.astype(np.int64)).to(torch.long)
col = torch.from_numpy(adj.col.astype(np.int64)).to(torch.long)
edge_index = torch.stack([row, col], dim=0)


class TwitterDataset(InMemoryDataset):
    def __init__(self, transform=None):
        super(TwitterDataset, self).__init__('.', transform, None, None)

        data = Data(edge_index=edge_index)
        
        data.num_nodes = g.number_of_nodes()
        
        # Features 
        data.x = torch.from_numpy(features).type(torch.float32)
        
        # Etiquetas
        y = torch.from_numpy(labels).type(torch.long)
        data.y = y.clone().detach()
        
        data.num_classes = 2
        
        n_nodes = g.number_of_nodes()
        
        # create train and test masks for data
        train_mask = torch.zeros(n_nodes, dtype=torch.bool)
        train_mask[train_idx] = True
        data['train_mask'] = train_mask

        self.data, self.slices = self.collate([data])

    def _download(self):
        return

    def _process(self):
        return

    def __repr__(self):
        return '{}()'.format(self.__class__.__name__)